In [92]:
import warnings

warnings.filterwarnings('ignore')
warnings.simplefilter('ignore')

from pydataxm import *                           #Se realiza la importación de las librerias necesarias para ejecutar                        
from pydataxm.pydataxm import ReadDB as apiXM 
import datetime as dt                            
import pandas as pd
from pathlib import Path
import os

from google.cloud import bigquery
from google.oauth2 import service_account

In [55]:
FechaIni=dt.date(2025, 8,1)
FechaFin=dt.date(2025, 8, 31)
sFechaFin = FechaFin.strftime('%Y-%m-%d')
dias_mes = (FechaFin - FechaIni).days + 1

In [93]:
df_RecIni= apiXM.request_data(pydataxm.ReadDB(),    #Se indica el objeto que contiene el serivicio
                        "ListadoRecursos",                #Se indica el nombre de la métrica tal como se llama en el campo metricID
                        "Sistema",                 #Campo que indica si es Sistema, Agente, Recurso, Comercializador
                        FechaIni,       #Corresponde a la fecha inicial de la consulta
                        FechaFin)      #Corresponde a la fecha final de la consulta
df_RecIni['Values_Code']=df_RecIni['Values_Code'].astype(str)


In [95]:
df_RecIni.to_csv('Recursos.csv')

In [8]:
df_RecIni= apiXM.request_data(pydataxm.ReadDB(),    #Se indica el objeto que contiene el serivicio
                        "ListadoRecursos",                #Se indica el nombre de la métrica tal como se llama en el campo metricID
                        "Sistema",                 #Campo que indica si es Sistema, Agente, Recurso, Comercializador
                        FechaIni,       #Corresponde a la fecha inicial de la consulta
                        FechaFin)      #Corresponde a la fecha final de la consulta
df_RecIni['Values_Code']=df_RecIni['Values_Code'].astype(str)

# Units kWh, Descarga de la disponibilidad real
df_CENIni= apiXM.request_data(pydataxm.ReadDB(),    #Se indica el objeto que contiene el serivicio
                        "CapEfecNeta",                #Se indica el nombre de la métrica tal como se llama en el campo metricID
                        "Recurso",                 #Campo que indica si es Sistema, Agente, Recurso, Comercializador
                        FechaIni,       #Corresponde a la fecha inicial de la consulta
                        FechaFin)      #Corresponde a la fecha final de la consulta
df_CENIni['Code']=df_CENIni['Code'].astype(str)

# Units kWh, Descarga de la disponibilidad real
df_GenRealIni= apiXM.request_data(pydataxm.ReadDB(),    #Se indica el objeto que contiene el serivicio
                        "Gene",                #Se indica el nombre de la métrica tal como se llama en el campo metricID
                        "Recurso",                 #Campo que indica si es Sistema, Agente, Recurso, Comercializador
                        FechaIni,       #Corresponde a la fecha inicial de la consulta
                        FechaFin)      #Corresponde a la fecha final de la consulta
df_GenRealIni['Values_code']=df_GenRealIni['Values_code'].astype(str)

In [91]:
df_CEN=df_CENIni.copy()
df_Rec=df_RecIni.copy()
df_GReal=df_GenRealIni.copy()

# df_CEN.fillna(0, inplace=True)
df_CEN['Value']=df_CEN['Value']/1000
df_CEN.rename(columns={'Value': 'CEN'}, inplace=True)
last_date = df_CEN['Date'].iloc[-1]
df_CEN=df_CEN[(df_CEN.Date==last_date)]



if 'Date' in df_Rec.columns:
    df_Rec=df_Rec.drop('Date',axis=1)

df_GReal=df_GReal.fillna(0)
l_per=[]
for i in range(1,25):
    if i<10:
        df_GReal=df_GReal.rename(columns={'Values_Hour0' + str(i) : str(i) })
    else:
        df_GReal=df_GReal.rename(columns={'Values_Hour' + str(i) : str(i) })

    df_GReal[str(i)]=df_GReal[str(i)]/1000
    l_per.append(str(i))

# Converti columnas de periodos en una sola columna
df_GReal = df_GReal.melt(id_vars=['Values_code', 'Date'], value_vars=l_per,var_name='periodo',value_name='Valor')
df_GReal['periodo'] = df_GReal['periodo'].astype(int)

df_GReal=df_GReal.groupby(['Values_code'])['Valor'].sum().round(2).reset_index()
df_GReal

df_Rec=df_Rec[df_Rec.Values_Type=='SOLAR']
df_Rec=df_Rec.merge(df_CEN,left_on=['Values_Code'],right_on=['Code'],how='inner')[['Values_Name','Code','Values_Type','Values_Disp','CEN']]
# df_Rec['CEN']=(df_Rec['CEN']*24).round(2)
df_Rec=df_Rec.merge(df_GReal,left_on=['Code'],right_on=['Values_code'],how='inner')[['Values_Name','Code','Values_Type','Values_Disp','CEN','Valor']]
df_Rec['Valor']=df_Rec['Valor']/dias_mes
df_Rec['FactorPlanta']=(100*df_Rec['Valor']/(df_Rec['CEN']*24)).round(2)
df_Rec=df_Rec.rename(columns={'Values_Name':'Planta','Valor':'Generación','Values_Disp':'Tipo_Despacho'})
df_Rec=df_Rec[['Planta','Tipo_Despacho','CEN','Generación','FactorPlanta']]
df_Rec



,Planta,Tipo_Despacho,CEN,Generación,FactorPlanta
0,LATAM SOLAR LA LOMA,DESPACHADO CENTRALMENTE,150.00000,826.596774,22.96
1,BOSQUES SOLARES DE LOS LLANOS 5,NO DESPACHADO CENTRALMENTE,17.90000,91.781935,21.36
2,BOSQUES SOLARES DE LOS LLANOS 4,NO DESPACHADO CENTRALMENTE,19.90000,101.524839,21.26
3,AGPE FERCH2,NO DESPACHADO CENTRALMENTE,0.00675,0.014516,8.96
4,GRANJA SOLAR EL SALADO,NO DESPACHADO CENTRALMENTE,0.10000,0.482581,20.11
...,...,...,...,...,...
75,GD BLANCA ENERGY II,NO DESPACHADO CENTRALMENTE,0.10000,0.212903,8.87
76,EL PASO,DESPACHADO CENTRALMENTE,68.00000,482.346774,29.56
77,GUAYEPO,DESPACHADO CENTRALMENTE,370.00000,2058.928065,23.19
78,LA MATA,DESPACHADO CENTRALMENTE,80.00000,495.420323,25.80
